In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt                                            
import matplotlib as mpl                                                    
mpl.rcParams['axes.unicode_minus'] = False                                  
#plt.rcParams('font.family') = 'RIDIBatang'                          
#plt.rcParams('font.size') = 16                                             
import matplotlib.font_manager as fm
font = 'C:\\Windows\\Fonts\\RIDIBatang.otf'
fontprop = fm.FontProperties(fname=font, size=16).get_name()
plt.rc('font', family = 'RIDIBatang')
plt.rc('font', size = 16)
import seaborn as sns    
from sklearn import datasets

Instructions for updating:
non-resource variables are not supported in the long term


***
데이터 전처리
***
우리가 실제로 접하게 되는 데이터는 Scikit-learn에서 제공하는 데이터와는 다르게 머신러닝을 적용하기 전에 데이터를 가공해야 하는 경우가 대부분이다.  
이를 데이터 전처리 또는 피쳐 엔지니어링 등으로 부른다.  

***
결측치(missing value) 처리
***
Dataset의 일부에 데이터가 존재하지 않을 경우 결측치라고 부르며, 머신러닝 알고리즘을 적용하기 전에 결측치를 처리하는 것이 좋다.

In [2]:
# 데이터프레임에 데이터가 존재하지 않는 결측치는 없지만 부적절한 값(이상치)이 존재한다.
# 나이에 1000이라는 값이 존재하는 것과 나머지 피쳐에 'unknown'이라는 값이 입력된 부적절한 값은 결측치로 처리하는 것이 좋다.
df = pd.DataFrame([
    [42, 'male', 12, 'reading', 'class2'],
    [35, 'unknown', 3, 'cooking', 'class1'],
    [1000, 'female', 7, 'cycling', 'class3'],
    [1000, 'unknown', 21, 'unknown', 'unknown']
])
# 나이, 성별, 태어난달, 취미, 레이블
df.columns = ['Age', 'Gender', 'Birth_M', 'Hobby', 'Label']
df

,Age,Gender,Birth_M,Hobby,Label
0,42,male,12,reading,class2
1,35,unknown,3,cooking,class1
2,1000,female,7,cycling,class3
3,1000,unknown,21,unknown,unknown


In [3]:
# unique() 메소드로 각 시리즈의 유일한 값을 확인할 수 있다.
print('Age:', df.Age.unique())
print('Gender:', df.Gender.unique())
print('Birth_M:', df.Birth_M.unique())
print('Hobby:', df.Hobby.unique())
print('Label:', df.Label.unique())

Age: [  42   35 1000]
Gender: ['male' 'unknown' 'female']
Birth_M: [12  3  7 21]
Hobby: ['reading' 'cooking' 'cycling' 'unknown']
Label: ['class2' 'class1' 'class3' 'unknown']


In [4]:
# 부적절한 데이터 값을 결측치로 바꾸기 위해 np.NaN을 사용한다.
df.loc[df.Age > 150, 'Age'] = np.NaN
df.loc[df.Gender == 'unknown', 'Gender'] = np.NaN
df.loc[df.Birth_M > 12, 'Birth_M'] = np.NaN
df.loc[df.Hobby == 'unknown', 'Hobby'] = np.NaN
df.loc[df.Label == 'unknown', 'Label'] = np.NaN
df

,Age,Gender,Birth_M,Hobby,Label
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3
3,NaN,NaN,NaN,NaN,NaN


In [5]:
# Dataset의 각 열의 결측치 갯수를 확인한다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Age      2 non-null      float64
 1   Gender   2 non-null      object 
 2   Birth_M  3 non-null      float64
 3   Hobby    3 non-null      object 
 4   Label    3 non-null      object 
dtypes: float64(2), object(3)
memory usage: 288.0+ bytes


In [6]:
df.isna().sum()

Age        2
Gender     2
Birth_M    1
Hobby      1
Label      1
dtype: int64

***
결측치를 처리하는데 주로 쓰이는 방법은 결측치를 삭제하거나 다른 값으로 변경하는 방법이 있다.
***

In [7]:
# dropna() 메소드로 결측치를 삭제할 수 있다.
# axis 속성을 생략하거나 0으로 지정하면 결측치가 하나라도 포함된 모든 행을 삭제한다.
# inplace 속성을 True를 지정하면 실행 결과를 대입하지 않아도 실행결과를 즉시 반영한다.
df.dropna() #df.dropna(axis=0)

,Age,Gender,Birth_M,Hobby,Label
0,42.0,male,12.0,reading,class2


In [8]:
# axis 1로 지정하면 결측치가 하나라도 포함된 모든 열을 삭제한다. ; 데이터가 NaN이 들어있으면 열단위로 지운다.
df.dropna(axis=1) 

""
0
1
2
3


In [9]:
# how 속성을 'all'로 지정하면 모든 값이 결측치인 행을 삭제한다.
df.dropna(how='all')

,Age,Gender,Birth_M,Hobby,Label
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3


In [10]:
# thresh 속성에 숫자를 지정하면 결측치를 제외한 갯수가 숫자보다 작은 행을 삭제한다.
df.dropna(thresh=5)

,Age,Gender,Birth_M,Hobby,Label
0,42.0,male,12.0,reading,class2


In [11]:
# subset 속성에 열이름을 지정하면 특정 열에 결측치가 존재하는 행을 삭제한다.
df.dropna(subset=['Gender'])

,Age,Gender,Birth_M,Hobby,Label
0,42.0,male,12.0,reading,class2
2,NaN,female,7.0,cycling,class3


In [12]:
#fillna() 메소드로 결측치를 대체할 값을 지정할 수 있다.
df.fillna(0)

,Age,Gender,Birth_M,Hobby,Label
0,42.0,male,12.0,reading,class2
1,35.0,0,3.0,cooking,class1
2,0.0,female,7.0,cycling,class3
3,0.0,0,0.0,0,0


In [13]:
# fillna() 메소드의 value 속성으로 결측치를 대체할 값이 저장된 딕셔너리를 지정해서 열별로 결측치를 대체할 수 있다.
# 열 이름을 key로 하고 결측치를 대체할 값을 value로 하는 딕셔너리를 지정한다.
alter_values = {'Age' : 0, 'Gender' : '0', 'Birth_M' : 0.0, 'Hobby' : 'Nothing', 'Label' : 'class4'}
df.fillna(value=alter_values, inplace=True)

***
기계 처리에 적합하도록 클래스 레이블 설정
***
Label 열을 보면 문자열(Object) 형태인데, 기계처리에 적합하도록 하기위해 문자열을 상수(int)로 바꿔야 하는 경우가 있다.  


***
Scikit-learn을 사용하지 않고 레이블 설정하기
***

In [14]:
# 라벨링할 레이블만 얻어와서 오름차순으로 정렬한다.
label_df = df.Label.values
print(label_df)
label_df.sort()# 오름차순 정렬
#label_df[::-1] # 내림차순 정렬
print(label_df)


['class2' 'class1' 'class3' 'class4']
['class1' 'class2' 'class3' 'class4']


In [15]:
# 반복문을 실행해서 정렬된 레이블을 Key로 일련번호를 value로 하는 딕셔너리를 만든다.
label_dict = dict()
for key, value in enumerate( label_df ) :
     #print(idx, val)
     label_dict[value] = key
print(label_dict)

{'class1': 0, 'class2': 1, 'class3': 2, 'class4': 3}


In [16]:
#### replace() 메소드를 이용해서 딕셔너리에 저장된 데이터로 레이블을 라벨링한다.
df['Label_1'] = df.Label.replace(label_dict)
df

,Age,Gender,Birth_M,Hobby,Label,Label_1
0,42.0,male,12.0,reading,class1,0
1,35.0,0,3.0,cooking,class2,1
2,0.0,female,7.0,cycling,class3,2
3,0.0,0,0.0,Nothing,class4,3


***
Scikit-learn으로 레이블 설정하기
***

In [17]:
# 레이블 라벨링을 하기 위해 import 한다.
from sklearn.preprocessing import LabelEncoder

In [18]:
encoded_Label = LabelEncoder() #라벨링을 실행할 객체를 만든다.
unLabel = df.Label.values #라벨링할 데이터를 얻어온다.
print(unLabel)

#new_Label = encoded_Label.fit(unLabel) # 학습시킨다.
#new_Label = new_Label.transform(unLabel) # 학습한 결과를 적용시킨다.
# fit_transform() 메소드는 학습(fit)과 적용(transform)을 한번에 실행한다.
new_Label = encoded_Label.fit_transform(unLabel)
print(new_Label)

# 라벨링한 결괄르 데이터프레임에 넣어준다.
df['Label_2'] = new_Label
df

['class1' 'class2' 'class3' 'class4']
[0 1 2 3]


,Age,Gender,Birth_M,Hobby,Label,Label_1,Label_2
0,42.0,male,12.0,reading,class1,0,0
1,35.0,0,3.0,cooking,class2,1,1
2,0.0,female,7.0,cycling,class3,2,2
3,0.0,0,0.0,Nothing,class4,3,3


***
원-핫 인코딩(One-Hot Encoding)
***
원-핫 인코딩은 클래스를 라벨링하는 또다른 방법으로 오직 0과 1만 사용한 벡터를 이용해서 데이터 값을 나타내는 것이다.

***
* Pandas를 이용한 원-핫 인코딩
***

In [19]:
pd.get_dummies(df.Label)

,class1,class2,class3,class4
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1


In [20]:
pd.get_dummies(df.Age)

,0.0,35.0,42.0
0,0,0,1
1,0,1,0
2,1,0,0
3,1,0,0


In [21]:
# 위 방법에서는 길이가 4인 벡터를 이용해서 클래스(레이블)를 구분했다.
# drop_first 속성값을 True로 지정하면 벡터 길이를 하나 줄인 벡러를 이용해 원-핫인코딩을 할 수 있다.
# drop_first=True 속성을 지정하면 3개의 0과 1의 조합으로 4개의 클래스가 구분되는 것을 확인할 수 있다.
pd.get_dummies(df.Label, drop_first=True)

,class2,class3,class4
0,0,0,0
1,1,0,0
2,0,1,0
3,0,0,1


***
Scikit-learn을 이용한 원-핫인코딩
***

In [22]:
# 원-핫 인코딩을 하기위해 import 한다.
from sklearn.preprocessing import OneHotEncoder

In [23]:
one_hot = OneHotEncoder() # 원-핫인코딩을 실행할 객체를 만든다.
label = df[['Label']] # 원-핫 인코딩을 실행할 데이터를 얻어온다.
print(label)

#new_Label = one_hot.fit(label) # 학습시킨다. 원-핫 인코딩을 실행한다.
#new_Label = one_hot.transform(label) # 학습한 결과를 적용시킨다.
new_Label = one_hot.fit_transform(label)
print(new_Label) # 결과를 출력하면 희소 행렬로 출력된다.
print(type(new_Label))

print(new_Label.toarray())
print(type(new_Label.toarray()))

    Label
0  class1
1  class2
2  class3
3  class4
  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
<class 'scipy.sparse.csr.csr_matrix'>
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
<class 'numpy.ndarray'>


***
tensorflow를 이용한 원-핫인코딩
***

In [24]:
# 원-핫인코딩을 하기 위해 import 한다.
from tensorflow.keras.utils import to_categorical

In [25]:
label = df[['Label_2']] # 원-핫 인코딩을 실행할 데이터를 얻어온다.
new_Label = to_categorical(label) #원-핫 인코딩을 실행한다. 인수로 반드시 숫자를 넘겨야한다.
print(new_Label)
print(type(new_Label))


[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
<class 'numpy.ndarray'>


 ***
 데이터 스케일링(Data scaling) -> 정규화
 ***
 피쳐는 제각기 다른 단위를 가지기때문에 이로인해 숫자 자체의 크기 차이가 발생한다.  
 데이터 스케일링은 데이터 값이 단위의 영향을 받지 않도록 변형하는 것을 의미한다.  
 전체 데이터를 학습데이터와 테스트 데이터로 나누고 스케일링의 기준이 되는 값을 구할 때는 학습데이터만으로 구한다.

***
-  표준화(Standard) 스케일링 
***
대표적인 데이터 스케일링 방법으로 평균이 0, 표준편차가 1이 되도록 변경하는 방법이다. -> 표준 정규 분포;평균 0, 표준편차 1

$$\frac{x_i - \bar x}{\sigma}$$

기존 데이터값에서 평균을 뺀 후, 표준편차로 나눈다.

In [26]:
# 표준화 스케일링을 하기 위해 import 한다.
from sklearn.preprocessing import StandardScaler

In [27]:
std = StandardScaler() # 표준화 스케일링 객체를 선언한다.
#표준화 스케일러를 적용할 데이터를 fit() 메소드로 학습시킨 후 transform() 메소드로 적용시킨다.
y = df[['Birth_M']]
print(y)
print('----------')
std.fit(y)
x_std = std.transform(y)
print(x_std)

   Birth_M
0     12.0
1      3.0
2      7.0
3      0.0
----------
[[ 1.44444444]
 [-0.55555556]
 [ 0.33333333]
 [-1.22222222]]


In [28]:
# 실제로 표준화 스케일링 후 데이터가 평균이 0, 표준편차가 1인지 확인한다.
print('평균 -> {:.1f}, 표준편차 -> {:1f}'.format(np.mean(x_std), np.std(x_std)))

평균 -> -0.0, 표준편차 -> 1.000000


***
로버스트 스케일링
***
표준화 스케일리을 변형한 방법으로 중앙값(median)과 사분위수(quantile)을 사용한다. 극단값의 영향을 거의 받지 않는 다는 장점이 있다..

$$\frac{x_i - q_2}{q_3 - q_1}$$

위 식에서 q<sub>1</sub>는 2사분위수(중위수), a<sub>3</sub>는 3분위수를 의미한다.                                                                                                                                                                                             

In [29]:
# 로버스트 스케일링을 하기 위해 import 한다.
from sklearn.preprocessing import RobustScaler

In [30]:
robust = RobustScaler() # 로버스트 스케일링 객체를 선언한다.
#로버스트 스케일러를 적용할 데이터를 fit() 메소드로 학습시킨 후 transform() 메소드로 적용시킨다.
y = df[['Birth_M']]
robust.fit(y) # 로버스트 스케일링을 학습시킨다.
x_robust = robust.transform(y) # 학습 결과를 적용시킨다.
print(x_robust)

[[ 1.16666667]
 [-0.33333333]
 [ 0.33333333]
 [-0.83333333]]


***
최소-최대(min-max) 스케일링
***
$$\frac{x_i - min(x)}{max(x) - min(x)}$$

데이터 값의 범위를 최소값(0)과 최대값(1) 사이의 값으로 범위를 제한한다.

In [31]:
#최소-최대 스케일리을 하기 위해 import 한다.
from sklearn.preprocessing import MinMaxScaler

In [32]:
min_max = MinMaxScaler() # 로버스트 스케일링 객체를 선언한다.
#로버스트 스케일러를 적용할 데이터를 fit() 메소드로 학습시킨 후 transform() 메소드로 적용시킨다.
y = df[['Birth_M']]
#min_max.fit(y) # 로버스트 스케일링을 학습시킨다.
#x_min_max = min_max.transform(y) # 학습 결과를 적용시킨다.
x_min_max = min_max.fit_transform(y)
print(x_min_max)

[[1.        ]
 [0.25      ]
 [0.58333333]
 [0.        ]]


***
노멀(Normal Scaling) 스케일링
***
벡터의 유클리드 거리(두 점 사이의 거리 계산에 사용)가 1이 되도록 데이터값을 변경한다.  
주로, 벡터의 길이는 상관없고 방향(각도)만을 고려할 때 사용한다.  
표준화, 로버스트, 최소-최대 스케일링은 열 기준인 것과 다르게 노멀 스케일링은 행 기준이다.  

$$\frac{x_i}{\sqrt{x_i^2 + y_i^2 + z_i^2}}$$


In [33]:
#노멀 스케일링을 하기 위해 import 한다.
from sklearn.preprocessing import Normalizer

In [34]:
normal = Normalizer() # 로버스트 스케일링 객체를 선언한다.
#로버스트 스케일러를 적용할 데이터를 fit() 메소드로 학습시킨 후 transform() 메소드로 적용시킨다.
y = df[['Age', 'Birth_M']]
#normal.fit(y) # 로버스트 스케일링을 학습시킨다.
#x_normal = normal.transform(y) # 학습 결과를 적용시킨다.
x_normal = normal.fit_transform(y)
print(x_normal)

[[0.96152395 0.27472113]
 [0.99634665 0.08540114]
 [0.         1.        ]
 [0.         0.        ]]


***
데이터 스케일링 과정에서 fit() 메소드는 학습 데이터에 대해서만 사용하며, 테스트 데이터에는 fit() 메소드를 사용하지 않고  
transform() 메소드만 사용한다.   
그 이유는 스케일러는 학습 데이터를 기반으로 만들어야하기 때문이다.  
만약, 학습 데이터와 테스트 데이터에 fit() 메소드를 실행해서 각각이 데이터로 스케일링을 한다면 학습 데이터와 테스트 데이터의 분포가 다르기때문에  
학습 데이터와 테스트 데이터의 스케일링 범위 및 파라미터가 달라진다.
***

from skleaarn.preprocessing import StandardScaler
std = StandarScaler()  

std.fit(x_train) # 학습 데이터로 스케일러를 학습시킨다.  
x_train_std = std.transform(x_train) # 학습 데이터에 스케일러를 적용시킨다.  

또는   

x_train_std = std.fit_transform(x_train)  
방법을 사용해서 스케일러를 학습시킨 후 적용한다.

테스트 데이터는 fit() 메소드를 사용하지 않고 학습 데이터로 학습된 스케일러에 테스트 데이터를 적용만 시킨다.  
x_test_std = std.transform(x_test)  


***